In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

#input_ids = tokenizer(prompt, return_tensors="pt").input_ids

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [3]:
import torch
prompts = ["Today I believe we can finally","Hello my name is John,","In the mysterious night","Love or hate it, Pizza is ","My dog jumped out"]
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [4]:

torch.random.manual_seed(42)
#Top- K sampling
generation_params = {
    "greedy_search": {"do_sample": False, "max_length": 50},
    "beam_search": {"num_beams": 5, "max_length": 50, "early_stopping": True},
    "top_k_sampling": {"do_sample": True, "top_k": 50, "max_length": 50},
    "top_p_sampling": {"do_sample": True, "top_p": 0.8, "max_length": 50}
}

In [5]:
def calculate_perplexity(input_text):
    encodings = tokenizer(input_text, return_tensors="pt").to(device)
    max_length = model.config.n_positions
    input_ids = encodings.input_ids
    target_ids = input_ids.clone()
    target_ids[:, :-1] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)
        neg_log_likelihood = outputs.loss

    return torch.exp(neg_log_likelihood).item()

In [6]:
perplexities = {}
for prompt in prompts:
  for strategy_name, params in generation_params.items():
      input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
      generated_outputs = model.generate(input_ids, **params, pad_token_id=tokenizer.eos_token_id)
      generated_text = tokenizer.decode(generated_outputs[0], skip_special_tokens=True)
      perplexity = calculate_perplexity(generated_text)
      perplexities[strategy_name] = perplexity
      print(f"Strategy: {strategy_name}\nGenerated Text: {generated_text}\nPerplexity: {perplexity}\n")

Strategy: greedy_search
Generated Text: Today I believe we can finally get to the point where we can make a difference in the lives of the people of the United States of America.

I believe that we can make a difference in the lives of the people of the United States of
Perplexity: 1.3299473524093628

Strategy: beam_search
Generated Text: Today I believe we can finally get to the point where we can make the world a better place," he said.

"We can make it a better place for all of us. We can make it a better place for all of us.
Perplexity: 2.0440218448638916

Strategy: top_k_sampling
Generated Text: Today I believe we can finally begin to take back the earth, and this is the first step towards making it a better place."

When contacted for her opinion, the girl's parents told The Guardian she had not visited the United States since 2014
Perplexity: 97.30875396728516

Strategy: top_p_sampling
Generated Text: Today I believe we can finally see an end to a system that is fundamentally fl